# MSD - List
## Info
* Auf einer Seite wurde die letzte Zeile nicht korrekt erkannt. Deshalb wird empty1 in total kopiert

In [4]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib
import pikepdf

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Unlock PDF
Dieser Teil entfernt den Passwortschutz, damit das PDF in HTML exportiert werden kann

In [5]:
pdf = pikepdf.open('2019_Final Disclosure Report.pdf')
pdf.save('unlocked.pdf')

## Import PDF

In [6]:
# Read pdf into DataFrame
df_list = tabula.read_pdf("unlocked.pdf", pages='all', lattice=True, multiple_tables=True)
df = pd.concat(df_list)

## Format Table

In [7]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'empty1', 'total']

#Shift
df_export[df_export['name'].isna()] = df_export[df_export['name'].isna()].shift(-1, axis='columns')

#Add Type
df_export = df_export.reset_index(drop=True)
index_hcp = df_export[df_export['name'].str.contains("OTHER, NOT INCLUDED ABOVE", na=False)].index[0]
df_export['type'] = np.where(df_export.index < index_hcp, 'hcp', 'hco')

#Copy empty1 to total, if not empty
df_export.loc[df_export['empty1'].notna(), 'total'] = df_export['empty1']

#Remove rows which have no values
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export.reset_index(drop=True)

#Remove empty
df_export.drop(columns=['empty1'], inplace=True)

#Remove \r
df_export = remove_carination(df_export, " ")

#Remove N/A
df_export = df_export.replace("N/A", "")

#Remove Name in Address
df_export['address'] = df_export.apply(lambda row: row['address'].replace(row['name'], '') , axis = 1)
df_export['address'] = df_export['address'].str.strip()

#Convert to Numbers
df_export = replace_comma_to_dot(df_export)
df_export = amounts_to_number(df_export)

#Add PLZ
df_export = add_plz(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

#shift Surname/Lastname
df_export = revert_name(df_export, ' ')

#Export
export_list(df_export, 'msd')

saved


In [8]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx', open=True)